In [1]:
import requests
from xml.etree import ElementTree
import pandas as pd
from pandas import DataFrame
!pip3 install lxml


URL = 'https://cbr.ru/scripts/XML_News.asp'
OUTPUT_FILE = '/opt/lab1/data.txt'
OUTPUT_FILE = 'data/output_data.csv'

response = requests.post(url=URL)
tree = ElementTree.fromstring(response.content)
str_xml = ElementTree.tostring(tree, encoding='utf-8')

print (str_xml[0:150])
ds = pd.read_xml(str_xml,  xpath=".//Item", encoding = 'utf-8')


ds.to_csv(OUTPUT_FILE, index=False)

b'<News><Item ID="2942"><Date>31.03.2023</Date><Url>/statistics/macro_itm/dkfs/#a_54699                                                                 '


In [2]:
### LAB2 
!pip install psycopg2-binary

In [3]:
import psycopg2
import os 

DB_HOST =  os.environ.get('DB_HOST','158.160.33.115')
DB_PORT = os.environ.get('DB_PORT', 5432) 
DB_USER = os.environ.get('DB_USER','ad_admin_user')
DB_PWD = os.environ.get('DB_PWD','top_secret')
DB_DBNAME = os.environ.get('DB_DBNAME','db_test')

print (DB_HOST)

158.160.33.115


In [4]:


conn = psycopg2.connect(f"host={DB_HOST} port={DB_PORT} dbname={DB_DBNAME} user={DB_USER} password={DB_PWD}")
print('Connected to: ', conn.get_dsn_parameters(), "\n")

cur = conn.cursor()
cur.execute("SELECT version();")
db_version = cur.fetchone()
print('PostgreSQL database version:',db_version)

schema_name = 'db_test'
table_name = 'msai_lab1_table'

check_table_sql = f'SELECT EXISTS ( \
   SELECT FROM pg_tables \
   WHERE  tablename  = \'{table_name}\' \
   );'

create_table_sql = f'Create  table {table_name} ( \
ID integer NOT  NULL, \
Date varchar(15), \
URL varchar(250), \
Title varchar(500),\
PRIMARY KEY (ID)\
);'

cur.execute(check_table_sql)
db_out = cur.fetchone()
print(db_out)
if db_out[0] == False:
    print ('Creating table with  sql: ', create_table_sql)
    create_result  = cur.execute(create_table_sql)
    print (create_result)
    
else:
    print (f'Table {table_name} already  exists')

conn.commit()

if 'ds' not  in globals() or  ds  is  None:
    print('Dataframe  is empty. Please  execute  first cell (lab1)')
else:
      for index, row in ds.iterrows():
       
        id_= row[0]
        date = row[1]
        url = row[2]
        title = row[3]
        
        insert_sql_query = f'INSERT INTO {table_name}  (ID, Date, URL,Title) \
        VALUES ({id_}, \'{date}\', \'{url}\',  \'{title}\' ) \
        ON CONFLICT (ID) DO UPDATE \
        SET Date = \'{date}\', \
        URL = \'{url}\', \
        Title = \'{title}\' '
    
  
        #print  (insert_sql_query)
        ins_result = cur.execute(insert_sql_query)
        #print (ins_result)
        
      conn.commit()        
      print(f'{index} Rows inserted  into {table_name}')
        

conn.close()

Connected to:  {'user': 'ad_admin_user', 'channel_binding': 'prefer', 'dbname': 'db_test', 'host': '158.160.33.115', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

PostgreSQL database version: ('PostgreSQL 15.2 (Debian 15.2-1.pgdg110+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit',)
(False,)
Creating table with  sql:  Create  table msai_lab1_table ( ID integer NOT  NULL, Date varchar(15), URL varchar(250), Title varchar(500),PRIMARY KEY (ID));
None
6812 Rows inserted  into msai_lab1_table


In [5]:
### Just  test cell 

conn = psycopg2.connect(f"host={DB_HOST} port={DB_PORT} dbname={DB_DBNAME} user={DB_USER} password={DB_PWD}")
print('Connected to: ', conn.get_dsn_parameters(), "\n")
cur = conn.cursor()
sel_query_sql = f'Select  *  from {table_name}'
sel_result  = cur.execute(sel_query_sql)
print (cur.fetchmany(10))

conn.close()
    

Connected to:  {'user': 'ad_admin_user', 'channel_binding': 'prefer', 'dbname': 'db_test', 'host': '158.160.33.115', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

[(2942, '31.03.2023', '/statistics/macro_itm/dkfs/#a_54699', 'Обзор банковской системы'), (2941, '31.03.2023', '/statistics/macro_itm/dkfs/#a_54698', 'Обзор кредитных организаций'), (2940, '31.03.2023', '/statistics/macro_itm/dkfs/#a_54697', 'Обзор центрального банка'), (6761, '31.03.2023', '/statistics/macro_itm/dkfs/#a_54696', 'Сезонно скорректированный ряд широкой денежной массы'), (6271, '31.03.2023', '/statistics/macro_itm/dkfs/#a_54695', 'Сезонно скорректированный ряд денежной массы (M2)'), (8377, '31.03.2023', '/statistics/ms/', 'Денежная масса М2 (национальное определение)'), (23055, '31.03.2023', '/statistics/macro_itm/svs/interactive/', 'Интерактивное п